In [1]:
%%bash

pip install -q kaggle
mkdir -p ~/.kaggle
cp drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
kaggle datasets download -d niraliivaghani/chatbot-dataset
unzip -qq chatbot-dataset.zip

100%|██████████| 4.57k/4.57k [00:00<00:00, 3.70MB/s]


In [19]:
!pip install numpy==1.22.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 40.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2


# ChatBot

In [1]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents\n")
print(len(classes), "classes\n")
print(len(words), "words\n")

405 documents

38 classes

263 words



In [9]:
training = []
output_empty = [0]*len(classes)
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [10]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [11]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[ 0.03205031  0.08531244 -0.06309965 ...  0.08186147 -0.05852892
   0.09290114]
 [ 0.09152608  0.09418271 -0.05921996 ... -0.11429558 -0.05916461
  -0.04671075]
 [-0.0646461   0.11523382  0.07645094 ... -0.12252311  0.02847259
   0.07154182]
 ...
 [ 0.01778197 -0.06809869 -0.01124866 ... -0.10066418 -0.08806796
  -0.09503219]
 [-0.10582235 -0.09668859  0.10338338 ... -0.04032451 -0.00364666
   0.12162335]
 [ 0.04961395  0.01031378  0.08745095 ... -0.04932238 -0.11255949
   0.12369314]]


In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
81/81 [==============================] - 2s 3ms/step - loss: 3.5817 - accuracy: 0.0716
Epoch 2/200
81/81 [==============================] - 0s 2ms/step - loss: 3.4358 - accuracy: 0.1358
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.2556 - accuracy: 0.1901
Epoch 4/200
81/81 [==============================] - 0s 3ms/step - loss: 2.9589 - accuracy: 0.2346
Epoch 5/200
81/81 [==============================] - 0s 2ms/step - loss: 2.7040 - accuracy: 0.3160
Epoch 6/200
81/81 [==============================] - 0s 3ms/step - loss: 2.4405 - accuracy: 0.3827
Epoch 7/200
81/81 [==============================] - 0s 3ms/step - loss: 2.2088 - accuracy: 0.4494
Epoch 8/200
81/81 [==============================] - 0s 2ms/step - loss: 2.0013 - accuracy: 0.4938
Epoch 9/200
81/81 [==============================] - 0s 2ms/step - loss: 1.8006 - accuracy: 0.5333
Epoch 10/200
81/81 [==============================] - 0s 2ms/step - loss: 1.6233 - accuracy: 0.5926
Epoch 11/

In [14]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

In [15]:
def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)

  bag = [0]*len(words) 
  for s in sentence_words:  
      for i,w in enumerate(words):
          if w == s: 
              bag[i] = 1
              if show_details:
                  print ("found in bag: %s" % w)
  return(np.array(bag))

In [16]:
def predict_class(sentence, model):
  p = bow(sentence, words,show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25

  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []

  for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

  return return_list

In [17]:
def getResponse(ints, intents_json):

  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
      if(i['tag']== tag):
          result = random.choice(i['responses'])
          break
  return result



In [18]:
def chatbot_response(text):
   ints = predict_class(text, model)
   res = getResponse(ints, intents)
   return res

In [19]:
start = True
while start:
  query = input('Enter Message:')
  if query in ['quit','exit','bye']:
    start = False
    continue
  try:
    res = chatbot_response(query)
    print(res)
  except:
    print('You may need to rephrase your question.')

Enter Message:Hey
1/1 [==============================] - 0s 107ms/step
Hi there, how can I help?
Enter Message:Who are yuop?
1/1 [==============================] - 0s 25ms/step
I am a Chatbot.
Enter Message:well, What are the things you can do?
1/1 [==============================] - 0s 25ms/step
You can ask me questions regarding college, and i will try to answer them
Enter Message:great!
1/1 [==============================] - 0s 25ms/step
Hi there, how can I help?
Enter Message:bye
